#Importation des librairies

In [96]:
#pipenv install pour installer les dependences
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
from urllib.request import urlopen
import json


#Importation des données


##dataset

In [2]:
data = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/78348f03-a11c-4a6b-b8db-2acf4fee81b1", sep="|", low_memory=False)

##Les villes de france

In [3]:
# https://sql.sh/736-base-donnees-villes-francaises
villes_france = pd.read_csv("villes_france.csv", sep=",",low_memory=False)

##Les fichiers json

In [72]:
france_departement_json = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"

#Nettoyage des données


##Pour le dataset

In [117]:
#pour nettoyer les données on va supprimer les colonnes où il n'y a pas de valeurs
clean_data = data.dropna(axis = 1, how = "all")

#on doit aussi supprimer les lignes vides si il y en a 
clean_data = clean_data.dropna(axis = 0, how = "all")

# le nettoyage des données est terminé 
#Par la suite, nous allons utiliser le dataframe clean_data



##Pour les villes

In [116]:
positions = villes_france.dropna(axis=0, how="all")

#Debut des visuels


##Ventes totales en France en 2023

In [13]:
ventes_totales_par_dates = clean_data
ventes_totales_par_dates["Date mutation"] = pd.to_datetime(ventes_totales_par_dates["Date mutation"], format='%d/%m/%Y')
ventes_par_date = ventes_totales_par_dates.groupby("Date mutation").size().reset_index(name="Nombre de ventes")
fig = px.bar(ventes_par_date, x='Date mutation', y='Nombre de ventes', title='Nombre de ventes par date')
fig.show()

c:\Users\Nizar\anaconda3\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:
#ici on va creer un nouveau dataframe pour facilite les suite car paris est divise en arrondissement
ventes_paris=clean_data[clean_data["Commune"].str.startswith("PARIS ")]
ventes_paris["Commune"] = "PARIS"
ventes_paris_par_departement = ventes_paris.groupby('Code departement').size().reset_index(name="Nombre de ventes")
ventes_paris_par_commune = ventes_paris.groupby('Commune').size().reset_index(name="Nombre de ventes")


#et on fait la meme avec marseille et lyon
ventes_marseille=clean_data[clean_data["Commune"].str.startswith("MARSEILLE ")]
ventes_marseille["Commune"] = "MARSEILLE"
ventes_marseille_par_departement = ventes_marseille.groupby('Code departement').size().reset_index(name="Nombre de ventes")
ventes_marseille_par_commune = ventes_marseille.groupby('Commune').size().reset_index(name="Nombre de ventes")

ventes_lyon=clean_data[clean_data["Commune"].str.startswith("LYON ")]
ventes_lyon["Commune"] = "LYON"
ventes_lyon_par_departement = ventes_lyon.groupby('Code departement').size().reset_index(name="Nombre de ventes")
ventes_lyon_par_commune = ventes_lyon.groupby('Commune').size().reset_index(name="Nombre de ventes")


##Les différents types de propriété vendus en France en 2023 

In [14]:
type_propriete = clean_data.groupby('Type local').size().reset_index(name="Nombre de ventes")
fig = px.pie(type_propriete, values='Nombre de ventes', names='Type local', title='nombre de ventes immobiliète par type de local en France en 2023')
fig.show()

##Ventes par département de France en 2023

In [9]:
ventes_par_departement = clean_data.groupby('Code departement').size().reset_index(name="Nombre de ventes")
fig = px.bar(ventes_par_departement, x='Code departement', y='Nombre de ventes', title='Nombre de ventes par departement')
fig.show()

##Nombre de pièces principales par type de local

In [126]:
nombre_piece_par_type = clean_data.groupby("Type local")["Nombre pieces principales"].mean().reset_index()
fig = px.bar(nombre_piece_par_type, x='Type local', y='Nombre pieces principales', title='Nombre de ventes par departement')
fig.show()

##Densité des ventes en France en 2023 par commune

In [ ]:
positions = positions.rename(columns={"Nom" : "Commune"})
positions = positions.rename(columns={"Departement" : "Code departement"})
ventes_par_commune = clean_data.groupby('Code departement').size().reset_index(name="Nombre de ventes")
ventes_par_commune = pd.concat([ventes_par_commune, ventes_paris_par_departement, ventes_marseille_par_departement, ventes_lyon_par_departement], ignore_index=True)
villes_et_positions = pd.merge(positions, ventes_par_commune, on="Code departement")
fig = px.density_mapbox(villes_et_positions, lat = "Latitude", lon = "Longitude", z='Nombre de ventes', radius=3, center=dict(lat=46.6035, lon=1.888334), zoom=3, mapbox_style = "open-street-map", title = 'Carte de densité des ventes immobilières en France')
fig.show()

##Prix du mètre carré moyen en France en 2023 par commune

In [ ]:
prix_metre_carre = pd.concat([clean_data, ventes_lyon, ventes_marseille, ventes_paris])
#on supprime les valeurs à superficie null 
prix_metre_carre = prix_metre_carre.dropna(subset="Surface reelle bati")
prix_metre_carre = prix_metre_carre[prix_metre_carre["Surface reelle bati"] > 0]
#on garde seulement les appartement et maisons car il y a des prix de dependences a 5000000 pour 0 m² a Paris par exemple. 
#Les prix des dependences sont pardois absurde

#nettoyage
prix_metre_carre = prix_metre_carre[prix_metre_carre["Type local"].isin(["Appartement", "Maison"])]
prix_metre_carre = prix_metre_carre[prix_metre_carre["Surface reelle bati"] > 0]
prix_metre_carre = prix_metre_carre[prix_metre_carre["Nombre de lots"] > 0]
prix_metre_carre = prix_metre_carre.dropna(axis=0, how="all")



#calcule du prix au mètre carré
prix_metre_carre["Valeur fonciere"] = prix_metre_carre["Valeur fonciere"].str.replace(',','.').astype(float)
prix_metre_carre["Prix metre carre"] = prix_metre_carre["Valeur fonciere"] / prix_metre_carre["Surface reelle bati"]



prix_metre_carre = prix_metre_carre.groupby("Code departement")["Prix metre carre"].mean().reset_index()
prix_metre_carre = prix_metre_carre.rename(columns={"Code departement": "code"})

with urlopen('https://france-geojson.gregoiredavid.fr/repo/departements.geojson') as response:
    counties = json.load(response)


fig = px.choropleth_mapbox(prix_metre_carre, geojson=counties, locations='code', color='Prix metre carre',
                           color_continuous_scale="Viridis",
                           range_color=(prix_metre_carre['Prix metre carre'].min(), prix_metre_carre['Prix metre carre'].max()),
                           mapbox_style="carto-positron",
                           zoom=4, center = {"lat": 46.6031, "lon": 1.8883},
                           opacity=0.5,
                           labels={'Prix metre carre':'Prix metre carre'},
                           featureidkey="feature.properties.code"
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


Pour mettre le code en HTML : jupyter nbconvert VotreNotebook.ipynb --to html


In [124]:
nombre_piece_par_type

,Type local,Nombre pieces principales
0,Appartement,2.543591
1,Dépendance,0.000000
2,Local industriel. commercial ou assimilé,0.000000
3,Maison,4.155217
